[모델 ANN + 선형회귀]
1. ANN 출력과 선형 회귀 출력의 가중합을 통해 최종 출력을 생성하는 모델
- 2개의 은닉층을 사용하는 ANN 모델구조
- 균등분포 (tf.random_uniform), sigmoid 활성화 함수 사용
- 선형 회귀 모델 y=W*x+b
2. 손실함수 : MSE
3. 옵티마이저 : Adam Optimizer (learning rate=0.001)
4. 학습 epoch=300
- batch size=10

In [81]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [82]:
def load_csv(file_path):

    # CSV 파일 로드 (첫 번째 행을 건너뜀, 앞에서부터 8개의 열만 불러옴)
    data = np.genfromtxt(file_path, delimiter=',', skip_header=1, usecols=range(8))
    
    # 'HAEMOGLOBINS'를 목표 변수로 설정 (두 번째 열)
    x_data = np.delete(data, 1, axis=1)  # 두 번째 열 제거 (입력 변수)
    y_data = data[:, 1].reshape(-1, 1)  # 두 번째 열이 목표 변수
    
    return x_data, y_data

def split_data(x_data, y_data):
    x_test = x_data[:400]
    y_test = y_data[:400]
    x_train = x_data[400:]
    y_train = y_data[400:]

    sc = StandardScaler()
    x_train = sc.fit_transform(x_train)
    x_test = sc.transform(x_test)
    
    return x_train, x_test, y_train, y_test

In [83]:
# 데이터 로드 및 분할
x_data, y_data = load_csv('data-ori.csv')
x_train, x_test, y_train, y_test = split_data(x_data, y_data)

In [84]:
learning_rate = 0.001
num_epochs = 300
batch_size = 10
input_size = x_train.shape[1]
hidden1_size = 64
hidden2_size = 32
output_size = 1

In [85]:
# 입력과 출력 플레이스홀더 정의
x = tf.placeholder(tf.float32, shape=[None, input_size])
y = tf.placeholder(tf.float32, shape=[None, output_size])

In [86]:
# ANN 모델 정의
def build_ANN(x):
    W1 = tf.Variable(tf.random_uniform([input_size, hidden1_size]))
    b1 = tf.Variable(tf.random_uniform([hidden1_size]))
    H1_output = tf.nn.sigmoid(tf.matmul(x, W1) + b1)

    W2 = tf.Variable(tf.random_uniform([hidden1_size, hidden2_size]))
    b2 = tf.Variable(tf.random_uniform([hidden2_size]))
    H2_output = tf.nn.sigmoid(tf.matmul(H1_output, W2) + b2)

    W_output = tf.Variable(tf.random_uniform([hidden2_size, output_size]))
    b_output = tf.Variable(tf.random_uniform([output_size]))
    
    logits = tf.matmul(H2_output, W_output) + b_output
    return logits

# 선형 모델 정의 (y = W * x + b)
def build_linear_model(x):
    W_linear = tf.Variable(tf.random_uniform([input_size, output_size]))
    b_linear = tf.Variable(tf.random_uniform([output_size]))
    linear_output = tf.matmul(x, W_linear) + b_linear  # 선형 회귀 모델
    return linear_output

In [87]:
# 모델 결합: ANN 출력과 선형 회귀 출력을 가중합
def build_combined_model(x):
    ann_output = build_ANN(x)
    linear_output = build_linear_model(x)
    
    # 가중합을 통해 두 출력 결합
    w_ann = tf.Variable(0.8, dtype=tf.float32)
    w_linear = tf.Variable(0.2, dtype=tf.float32)
    
    combined_output = w_ann * ann_output + w_linear * linear_output
    return combined_output

# 결합 모델 예측값
predicted_value = build_combined_model(x)

In [88]:
# 손실 함수 (Mean Squared Error)
loss = tf.reduce_mean(tf.square(predicted_value - y))

In [89]:
# Adam 옵티마이저 선언
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_step = optimizer.minimize(loss)

In [90]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    # 학습 반복
    for i in range(num_epochs):
        total_loss = 0
        num_batches = int(len(x_train) / batch_size)
        
        for j in range(num_batches):
            batch_x = x_train[j * batch_size:(j + 1) * batch_size]
            batch_y = y_train[j * batch_size:(j + 1) * batch_size]

            # 결합 모델 학습
            _, current_loss = sess.run([train_step, loss], feed_dict={x: batch_x, y: batch_y})
            total_loss += current_loss

        if i % 50 == 0:
            print(f"Epoch {i}/{num_epochs}, Training Loss: {total_loss/num_batches}")

    # 테스트 데이터로 성능 평가            
    y_pred = sess.run(predicted_value, feed_dict={x: x_test})
    
    # 예측값과 실제값의 차이 합 계산
    difference_sum = sum(abs(y_pred - y_test))
    print(f'차이 합: {difference_sum}')

Epoch 0/300, Training Loss: 1.5197032706398619
Epoch 50/300, Training Loss: 0.0005860543587506183
Epoch 100/300, Training Loss: 0.00041820247913943023
Epoch 150/300, Training Loss: 0.00031187367082686406
Epoch 200/300, Training Loss: 0.00024121473421233222
Epoch 250/300, Training Loss: 0.00022395627618657148
차이 합: [4.13688202]
